In [62]:
import pickle
import pandas as pd
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, train_test_split
import xgboost as xgb
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, EditedNearestNeighbours, NearMiss, ClusterCentroids
from sklearn.metrics import f1_score, classification_report, precision_recall_fscore_support
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.pipeline import Pipeline

In [63]:
with open('dataframe.pkl', 'rb') as file:
    df = pickle.load(file)

In [64]:
df.head()

,FlagImpaye,Montant,DateTransaction,CodeDecision,VerifianceCPT1,VerifianceCPT2,VerifianceCPT3,D2CB,ScoringFP1,ScoringFP2,...,DiffDateTr3,CA3TRetMtt,CA3TR,Heure,Year,Month,Day,Hour,Minute,Second
0,0,20.00,2017-02-01 07:32:14,1,0,0,0,551,0.000000,0.000000,...,4.0,20.00,0.00,27134,2017,2,1,7,32,14
1,0,20.00,2017-02-01 07:43:37,1,0,0,0,551,0.000000,0.000000,...,4.0,28.61,8.61,27817,2017,2,1,7,43,37
2,0,57.64,2017-02-01 07:47:38,1,0,0,0,549,0.000000,0.000000,...,4.0,57.64,0.00,28058,2017,2,1,7,47,38
3,0,54.29,2017-02-01 07:48:48,0,1,1,1,267,0.000000,0.000000,...,4.0,54.29,0.00,28128,2017,2,1,7,48,48
4,0,26.90,2017-02-01 08:13:27,1,0,0,0,549,0.003769,8.586333,...,4.0,59.15,32.25,29607,2017,2,1,8,13,27


In [65]:
df.dtypes

FlagImpaye               category
Montant                   float64
DateTransaction    datetime64[ns]
CodeDecision                int32
VerifianceCPT1              int32
VerifianceCPT2              int32
VerifianceCPT3              int32
D2CB                        int32
ScoringFP1                float64
ScoringFP2                float64
ScoringFP3                float64
TauxImpNb_RB              float64
TauxImpNB_CPM             float64
EcartNumCheq                int32
NbrMagasin3J                int32
DiffDateTr1               float64
DiffDateTr2               float64
DiffDateTr3               float64
CA3TRetMtt                float64
CA3TR                     float64
Heure                       int32
Year                        int32
Month                       int32
Day                         int32
Hour                        int32
Minute                      int32
Second                      int32
dtype: object

# DataFrame réduit :

L objectif est de réduire notre dataframe pour faire les différentes méthodes afin d avancer plus rapidement

In [66]:
# Séparez les données en fonction de la variable 'Flagimpaye'
df_pas_utilise, df_reduit = train_test_split(df, test_size=0.1, stratify=df['FlagImpaye'], random_state=42)

# Affichez la taille du sous-DataFrame de test pour vérifier
print("Taille du sous-DataFrame de test :", len(df_reduit))

Taille du sous-DataFrame de test : 464678


In [67]:
print("Taille du DataFrame:", len(df))

Taille du DataFrame: 4646773


# Apprentissage et test:

In [68]:
# Définition des dates limites pour les ensembles d'apprentissage et de test
train_inf = '2017-02-01'
train_sup = '2017-08-31'
test_inf = '2017-09-01'
test_sup = '2017-11-30'

Apprentissage:

In [69]:
train = df.loc[(df['DateTransaction'] >= train_inf) & (df['DateTransaction'] <= train_sup)]

In [70]:
x_train = train.drop(columns=['FlagImpaye'])
y_train = train['FlagImpaye']

Test:

In [71]:
test = df.loc[(df['DateTransaction'] >= test_inf) & (df['DateTransaction'] <= test_sup)]

In [72]:
x_test = test.drop(columns=['FlagImpaye'])
y_test = test['FlagImpaye']

In [73]:
y_test.value_counts()

FlagImpaye
0    730583
1      6485
Name: count, dtype: int64

# Modèles suppervisés

On enlève la colonne DateTransaction pour cette méthode car le type de données ne correspond pas

In [74]:
x_train = x_train.drop(columns=['DateTransaction'])

In [75]:
x_test = x_test.drop(columns=['DateTransaction'])

### Random Under-sampling

In [83]:
# Stratégie de validation croisée temporelle
tscv = TimeSeriesSplit(n_splits=5) 

In [84]:
# Paramètres à optimiser
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [10, 15],
    'classifier__min_samples_split': [2,5,10],
    'classifier__min_samples_leaf': [2,4],
    'undersampler__sampling_strategy': [0.1]
}

In [86]:
# Modèle de l'arbre de décision
gb_classifier = GradientBoostingClassifier()

In [87]:
# Initialisation du sous-échantillonneur aléatoire 
#rus = RandomUnderSampler(sampling_strategy=0.1)
rus = RandomUnderSampler()

# Sous-échantillonnage de l'ensemble d'entraînement et de test
#x_train_rus, y_train_rus = rus.fit_resample(x_train, y_train)
#x_test_rus, y_test_rus = rus.fit_resample(x_test, y_test)

In [88]:
# Pipeline
pipeline = Pipeline([
    ('undersampler', rus),
    ('classifier', gb_classifier)
])

### Random Under-sampling avec XGBoost

In [90]:
# GridSearchCV avec validation croisée temporelle
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1',
    cv=tscv,
    pre_dispatch='2*n_jobs',  # Ajoutez cette ligne pour résoudre certains problèmes
)
grid_search.fit(x_train, y_train)

ValueError: 
All the 120 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\imblearn\pipeline.py", line 292, in fit
    fit_params_steps = self._check_fit_params(**fit_params)
                       ^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Pipeline' object has no attribute '_check_fit_params'


In [57]:
#model = xgb.XGBClassifier()

# GridSearchCV avec validation croisée temporelle
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=tscv)
grid_search.fit(x_train, y_train)

ValueError: 
All the 120 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\imblearn\pipeline.py", line 292, in fit
    fit_params_steps = self._check_fit_params(**fit_params)
                       ^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Pipeline' object has no attribute '_check_fit_params'


In [32]:
# Afficher les meilleurs paramètres et la meilleure performance
print("Meilleurs paramètres : ", grid_search.best_params_)
print("Meilleur score F1 : ", grid_search.best_score_)

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [48]:
y_pred = model.predict(x_test_rus)

precision, recall, f1, _ = precision_recall_fscore_support(y_test_rus, y_pred, average='binary')

print(f'F1 Score: {f1}')

F1 Score: 0.8622568093385214


### Tomek Link

In [25]:
tl = TomekLinks()
x_train_tl, y_train_tl = tl.fit_resample(x_train, y_train)
x_test_tl, y_test_tl = tl.fit_resample(x_test, y_test)

### Tomek Link avec XGBoost

In [27]:
model = xgb.XGBClassifier()
model.fit(x_train_tl, y_train_tl)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [28]:
y_pred = model.predict(x_test_tl)

precision, recall, f1, _ = precision_recall_fscore_support(y_test_tl, y_pred, average='binary')

print(f'F1 Score: {f1}')

F1 Score: 0.8048568950563747


### Edited Nearest Neighbours

In [29]:
enn = EditedNearestNeighbours(n_neighbors=3)
x_train_enn, y_train_enn = enn.fit_resample(x_train, y_train)
x_test_enn, y_test_enn = enn.fit_resample(x_test, y_test)

### Edited Nearest Neighbours avec XGBoost

In [31]:
model = xgb.XGBClassifier()
model.fit(x_train_enn, y_train_enn)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [32]:
y_pred = model.predict(x_test_enn)

precision, recall, f1, _ = precision_recall_fscore_support(y_test_enn, y_pred, average='binary')

print(f'F1 Score: {f1}')

F1 Score: 0.8089500860585197


### NearMiss

In [33]:
x_train_nm, y_train_nm = NearMiss().fit_resample(x_train, y_train)
x_test_nm, y_test_nm = NearMiss().fit_resample(x_test, y_test)

### NearMiss avec XGBoost

In [36]:
model = xgb.XGBClassifier()
model.fit(x_train_nm, y_train_nm)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [37]:
y_pred = model.predict(x_test_nm)

precision, recall, f1, _ = precision_recall_fscore_support(y_test_nm, y_pred, average='binary')

print(f'F1 Score: {f1}')

F1 Score: 0.8872936109117013


## Decision Tree

In [109]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import f1_score

In [110]:
# Modèle de l'arbre de décision
dt_classifier = DecisionTreeClassifier()

### Decision tree avec Random Under-Sampling

In [112]:
# Entraîner le modèle sur l'ensemble d'entraînement
dt_classifier.fit(x_train_rus, y_train_rus)

# Faire des prédictions sur l'ensemble de test
y_pred = dt_classifier.predict(x_test_rus)

# Évaluer les performances du modèle
f1 = f1_score(y_test_rus, y_pred)
print(f"F1 Score: {f1}")

F1 Score: 0.8209606986899564


### Decision tree avec Tomek Link

In [113]:
# Entraîner le modèle sur l'ensemble d'entraînement
dt_classifier.fit(x_train_tl, y_train_tl)

# Faire des prédictions sur l'ensemble de test
y_pred = dt_classifier.predict(x_test_tl)

# Évaluer les performances du modèle
f1 = f1_score(y_test_tl, y_pred)
print(f"F1 Score: {f1}")

F1 Score: 0.666191155492154


### Decision tree avec Edited Nearest Neighbours

In [114]:
# Entraîner le modèle sur l'ensemble d'entraînement
dt_classifier.fit(x_train_enn, y_train_enn)

# Faire des prédictions sur l'ensemble de test
y_pred = dt_classifier.predict(x_test_enn)

# Évaluer les performances du modèle
f1 = f1_score(y_test_enn, y_pred)
print(f"F1 Score: {f1}")

F1 Score: 0.682643427741467


### Decision tree avec NearMiss

In [116]:
# Entraîner le modèle sur l'ensemble d'entraînement
dt_classifier.fit(x_train_nm, y_train_nm)

# Faire des prédictions sur l'ensemble de test
y_pred = dt_classifier.predict(x_test_nm)

# Évaluer les performances du modèle
f1 = f1_score(y_test_nm, y_pred)
print(f"F1 Score: {f1}")

F1 Score: 0.8342391304347826


Le meilleur est donc pour l'instant celui avec Tomek Link

## Gridsearch

In [38]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

### Gridsearch avec Random Under-Sampling

In [39]:
# Pipeline
pipeline = Pipeline([
    ('classifier', dt_classifier)
])

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__max_depth': [5, 10, 15],
    'classifier__min_samples_split': [5, 10],
    'classifier__min_samples_leaf': [2, 4]
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=5)
grid_search.fit(x_train_rus, y_train_rus)

# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score F1:", grid_search.best_score_)

Meilleurs paramètres: {'classifier__max_depth': 5, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 5}
Meilleur score F1: 0.8729769065625061


In [40]:
y_pred = grid_search.predict(x_test)

# Évaluer les performances du modèle
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1}")

F1 Score: 0.10013836042891733


In [41]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test, y_pred)

conf_matrix

array([[63136, 10307],
       [   99,   579]], dtype=int64)

### Gridsearch avec Tomek Link

In [42]:
# Pipeline
pipeline = Pipeline([
    ('classifier', dt_classifier)
])

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__max_depth': [5, 10, 15],
    'classifier__min_samples_split': [5, 10],
    'classifier__min_samples_leaf': [2, 4]
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=5)
grid_search.fit(x_train_tl, y_train_tl)

# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score F1:", grid_search.best_score_)

Meilleurs paramètres: {'classifier__max_depth': 5, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 5}
Meilleur score F1: 0.7780371337449985


In [43]:
y_pred = grid_search.predict(x_test)

# Évaluer les performances du modèle
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1}")

F1 Score: 0.8079242032730405


In [44]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test, y_pred)

conf_matrix

array([[73429,    14],
       [  209,   469]], dtype=int64)

### Gridsearch avec Edited Nearest Neighbours

In [46]:
# Pipeline
pipeline = Pipeline([
    ('classifier', dt_classifier)
])

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__max_depth': [5, 10, 15],
    'classifier__min_samples_split': [5, 10],
    'classifier__min_samples_leaf': [2, 4]
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=5)
grid_search.fit(x_train_enn, y_train_enn)

# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score F1:", grid_search.best_score_)

Meilleurs paramètres: {'classifier__max_depth': 5, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 10}
Meilleur score F1: 0.7784448743210616


In [47]:
y_pred = grid_search.predict(x_test)

# Évaluer les performances du modèle
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1}")

F1 Score: 0.8086206896551724


In [48]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test, y_pred)

conf_matrix

array([[73430,    13],
       [  209,   469]], dtype=int64)

### Gridsearch avec NearMiss

In [49]:
# Pipeline
pipeline = Pipeline([
    ('classifier', dt_classifier)
])

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__max_depth': [5, 10, 15],
    'classifier__min_samples_split': [5, 10],
    'classifier__min_samples_leaf': [2, 4]
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=5)
grid_search.fit(x_train_nm, y_train_nm)

# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score F1:", grid_search.best_score_)

Meilleurs paramètres: {'classifier__max_depth': 15, 'classifier__min_samples_leaf': 4, 'classifier__min_samples_split': 5}
Meilleur score F1: 0.9143329549436807


In [50]:
y_pred = grid_search.predict(x_test)

# Évaluer les performances du modèle
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1}")

F1 Score: 0.02968388589051658


In [51]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test, y_pred)

conf_matrix

array([[33233, 40210],
       [   62,   616]], dtype=int64)

Les modèles gridsearch ENN et TL sont les plus performants pour l'instant 